In [ ]:
"""

Author: Neeraj Jaiswal
Team Name: The Revengers
Team Members: Neeraj Jaiswal, Karanbir Singh, Komal Dadwal, Jatin Mahey, Sonu Choubey
Leaderboard Rank: 27

"""
import os
# This is the notebook used for making the inferences using the model trained.

clf_pipeline = joblib.load("../data/soil_classifier_pipeline.joblib")

def process_test_dataset_simple(test_csv_path, test_images_folder, clf_pipeline, output_csv_path):
    df_test = pd.read_csv(test_csv_path)  # Contains image_id only
    results = []

    for _, row in df_test.iterrows():
        image_id = row['image_id']
        image_path = os.path.join(test_images_folder, image_id)

        if not os.path.exists(image_path):
            print(f"Image not found: {image_path}")
            continue

        image = cv2.imread(image_path)
        if image is None:
            print(f"Could not read image: {image_path}")
            continue

        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        color = get_dominant_color(image)
        texture = get_texture_description(gray)

        R, G, B = parse_rgb(color)

        # Prepare input features as before, setting unknowns for drainage and fertility
        X_new = pd.DataFrame({
            'R': [R],
            'G': [G],
            'B': [B],
            'texture': [texture]
        })

        soil_pred = clf_pipeline.predict(X_new)[0]

        results.append({
            "image_id": image_id,
            "soil_type": soil_pred
        })

    result_df = pd.DataFrame(results)
    result_df.to_csv(output_csv_path, index=False)
    print(f"Saved predicted soil types to {output_csv_path}")
    return result_df

# Then call this function like:
data_dir = "../data/soil-classification"
test_csv_path = os.path.join(data_dir, "test_ids.csv")  # CSV with only image_id column
test_images_folder = os.path.join(data_dir, "test/")
output_csv_path = "soil_test_predictions.csv"

predicted_df = process_test_dataset_simple(test_csv_path, test_images_folder, clf_pipeline, "soil_test_predictions_simple.csv")
